In [5]:
import pymysql
import pandas as pd

In [3]:
conn = pymysql.connect(host="127.0.0.1", port=3306, db="apartment",
                user="root", passwd="1234")

In [4]:
cursor = conn.cursor()

In [27]:
seoul_df = pd.read_csv("data-files/seoul_apt.csv")

In [28]:
seoul_df.head(3)

,NO,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도
0,1,서울특별시 동대문구 이문동,84.972,202212,31,"76,500",2003
1,2,서울특별시 동작구 신대방동,84.940,202212,31,"86,500",2001
2,3,서울특별시 영등포구 양평동5가,59.940,202212,31,"77,000",1996


In [29]:
seoul_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12807 entries, 0 to 12806
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NO        12807 non-null  int64  
 1   시군구       12807 non-null  object 
 2   전용면적(㎡)   12807 non-null  float64
 3   계약년월      12807 non-null  int64  
 4   계약일       12807 non-null  int64  
 5   거래금액(만원)  12807 non-null  object 
 6   건축년도      12807 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 700.5+ KB


In [30]:
# 거래금액의 dtype을 object -> int
seoul_df["거래금액(만원)"] = seoul_df["거래금액(만원)"].map(lambda v : int(str(v).replace(",","")))
seoul_df["거래금액(만원)"].dtype

dtype('int64')

In [31]:
# sql에 table 생성
sql = """
    create table seoul
    (
        no varchar(5) primary key not null,
        place varchar(40) not null,
        area float not null,
        contract_year int,
        contract_day int,
        price int,
        create_year int
    )
    """
cursor.execute(sql)

0

In [32]:
# seoul_df를 list로 변환
seoul_list = seoul_df.values.tolist()

In [33]:
cursor.execute("delete from seoul")
conn.commit()

In [34]:
# sql에 data commit
sql = "insert into seoul values (%s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(sql, seoul_list)
conn.commit()

In [39]:
sql = "select * from seoul where place like %s"
cursor.execute(sql, '%{0}%'.format('노원구'))
nowon_list = cursor.fetchall()

In [40]:
nowon_list

(('10015', '서울특별시 노원구 상계동', 59.95, 202203, 19, 58000, 1998),
 ('10018', '서울특별시 노원구 상계동', 46.8, 202203, 19, 49900, 1989),
 ('10021', '서울특별시 노원구 하계동', 27.0, 202203, 18, 44800, 1988),
 ('10035', '서울특별시 노원구 상계동', 45.77, 202203, 18, 55000, 1988),
 ('10041', '서울특별시 노원구 상계동', 87.57, 202203, 18, 89000, 1988),
 ('10047', '서울특별시 노원구 중계동', 49.5, 202203, 18, 63000, 1990),
 ('10048', '서울특별시 노원구 상계동', 38.71, 202203, 18, 48500, 1989),
 ('1005', '서울특별시 노원구 중계동', 44.52, 202211, 28, 35000, 1992),
 ('10058', '서울특별시 노원구 상계동', 59.82, 202203, 18, 67500, 1998),
 ('10060', '서울특별시 노원구 중계동', 47.61, 202203, 18, 32800, 1988),
 ('10061', '서울특별시 노원구 공릉동', 59.94, 202203, 18, 81000, 2001),
 ('10065', '서울특별시 노원구 공릉동', 17.58, 202203, 18, 17000, 2018),
 ('10067', '서울특별시 노원구 월계동', 50.14, 202203, 18, 82000, 1986),
 ('10076', '서울특별시 노원구 상계동', 59.39, 202203, 18, 76700, 1988),
 ('10120', '서울특별시 노원구 상계동', 45.55, 202203, 17, 65000, 1988),
 ('1013', '서울특별시 노원구 중계동', 51.29, 202211, 27, 50800, 1989),
 ('10140', '서울특별시 노원구 월계동', 3

In [41]:
cursor.close()
conn.close()